# Natural Language Processing

Natural Language Processing (NLP) is a branch of artificial intelligence (AI) that deals with written and spoken language. You can use NLP to build solutions that extracting semantic meaning from text or speech, or that formulate meaningful responses in natural language.

## Using the Text Analytics Cognitive Service

Microsoft Azure *cognitive services* includes the *Text Analytics* service, which provides some out-of-the-box NLP capabilities, including the identification of key phrases in text, and the classification of text based on sentiment.

For example, suppose Adventure Works Cycles encourages customers to submit reviews for its cycle rental service. You could use the Text Analytics service to summarize the reviews by extracting key phrases, and determine which reviews are positive, and which are negative.

If you don't already have one, use the following steps to create a **Cognitive Services** resource in your Azure subscription:

1. In another browser tab, open the Azure portal (<a href='https://portal.azure.com' target='_blank'>https://portal.azure.com</a>), signing in with your Microsoft account.
2. Click the **&#65291;Create a resource** button, search for *Cognitive Services*, and create a **Cognitive Services** resource with the following settings:
    - **Name**: *Enter a unique name*.
    - **Subscription**: *Your Azure subscription*.
    - **Location**: *Any available location*.
    - **Pricing tier**: S0
    - **Resource group**: *Create a resource group with a unique name*.
3. Wait for deployment to complete. Then go to your cognitive services resource, and on the **Quick start** page, note the keys and endpoint. You will need these to connect to your cognitive services resource from client applications.

To use your cognitive services resource, client applications need its  endpoint and authentication key:

1. Copy the **Key1** for your resource and paste it in the code below, replacing **YOUR_COG_KEY**.
2. Copy the **endpoint** for your resource and and paste it in the code below, replacing **YOUR_COG_ENDPOINT**.
3. Run the code in the cell below by clicking its green <span style="color:green">&#9655</span> button.

In [ ]:
import os
from scripts import reviews_analysis
from azure.cognitiveservices.language.textanalytics import TextAnalyticsClient
from msrest.authentication import CognitiveServicesCredentials

# Set API configuration
cog_key = 'YOUR_COG_KEY'
cog_endpoint = 'YOUR_COG_ENDPOINT'

# Get a client for your text analytics cognitive service resource
text_analytics_client = TextAnalyticsClient(endpoint=cog_endpoint,
                                            credentials=CognitiveServicesCredentials(cog_key))

# Read the reviews in the /data/reviews folder
reviews_folder = os.path.join('data', 'reviews')
reviews = []
for file_name in os.listdir(reviews_folder):
    review_text = open(os.path.join(reviews_folder, file_name)).read()
    review = {"id": file_name, "language": "en", "text": review_text}
    reviews.append(review)

# Analyze the reviews
reviews_analysis.analyze_reviews(text_analytics_client, reviews)

> **Note**: If you're curious about the code used to retrieve the key phrases and sentiment scores from the Text Analytics Service, look at the **reviews_analaysis.py** file in the **scripts** folder.

## Using the Language Understanding Intelligence Service

Increasingly, we expect computers to be able to use AI in order to understand spoken or typed commands in natural language. For example, Adventure Works Cycles might want to implement office automation that enables employees to control the office environment using voice commands such as "switch on the light" or "put the fan on", and have an AI-powered device understand the command and take appropriate action.

### Create a LUIS Authoring Resource

The Microsoft cognitive services include the Language Understanding Intelligence Service (LUIS), which enables you to define *intents* that are applied to *entities* based on *utterances*. You can use your Cognitive Services resource to *publish* a LUIS app, but you must create a separate resource for *authoring* the app.

1. In another browser tab, open the Azure portal (<a href='https://portal.azure.com' target='_blank'>https://portal.azure.com</a>), signing in with your Microsoft account.
2. Click **+ Create a resource**, and search for *Language Understanding*.
3. In the list of services, click **Language Understanding**.
4. In the **Language Understanding** blade, click **Create**.
5. In the **Create** blade, select only **Authoring**. Then enter the following details and click **Create**
  * **Name**: A unique name for your service
  * **Subscription**: Your Azure subscription
  * **Resource Group**: The existing resource group you used previously
  * **Authoring location**: Any available location
  * **Authoring pricing tier**: F0
6. Wait for the service to be created.

### Create a LUIS App

To implement natural language understanding with LUIS, you create an app; and then add entities, intents, and utterances to define the commands you want the app to understand:

1. Open a new browser tab and navigate to the LUIS portal at <a href='https://preview.luis.ai/' target='_blank'>https://preview.luis.ai/</a>.
2. Sign in using the Microsoft account associated with your Azure subscription.
3. If this is the first time you have signed into LUIS, you may need to grant the app some permissions to access your account details. Then complete the *Welcome* steps by selecting the existing LUIS authoring resource you just created in your Azure subscription. 
4. Open the **My Apps** page, and select your subscription and LUIS authoring resource. Then click **Create** and create a new app with the following settings:
  - **Name**: Office Automation
  - **Culture**: English
  - **Description**: Adventure Works office automation
  - **Prediction resource**: *Your cognitive services resource*
5. If a panel with tips for creating an effective LUIS app is displayed, close it.

### Create an Entity

An *entity* is a thing that your language model can identify and do something with. In this case, your LUIS app will be used to control various *devices* in the office, such as lights or fans; so you'll create a *device* entity that includes a list of the types of device that you want the app to work with. For each device type, you'll create a sublist that identifies the name of the device (for example *light*) and any synonyms that might be used to refer to this type of device (for example *lamp*).

1. In the LUIS page for your app, in the pane on the left, click **Entities**. Then click **Create new entity**, and create a new entity named **device** of type **List** and click **Next**.
2. In the **Create a list entity** page, click *Add new sublist* and type **light**, then press ENTER.
3. After the **light** entity has been added, under **Synonyms**, click *Type in value*, and type **lamp** and press ENTER.
4. Add a second sublist named **fan** with the synonym **AC**.
5. Click **Create** to create the **device** entity with its **light** and **fan** sublists.

### Create Intents

An *intent* is an action you want to perform on one or more entities - for example, you might want to switch a light on, or turn a fan off. In this case, you'll define two intents: one to switch a device on, and another to switch a device off. For each intent, you'll specify sample *utterances* that indicate the kind of language used to indicate the intent.

1. In the pane on the left, click **Intents**. Then click **Create new intent**, and add an intent with the name **Switch On**.
2. In the **Utterances** page for the **Switch On** intent, type ***turn the light on*** and press **Enter** to add this utterance to the list.
3. In the list of utterances, in the *turn the light on* utterance, click the word "light", and assign it to the **device** entity.
4. Add a second utterance to the **Switch On** intent, with the phrase ***turn the fan on***. Then assign the word "fan" to the **device** entity.
5. In the pane on the left, click **Intents** and click **Create new intent**, to add a second intent with the name **Switch Off**.
6. In the **Utterances** page for the **Switch Off** intent, add the utterance ***turn the light off*** and assign the word "light" to the **device** entity.
7. Add a second utterance to the **Switch Off** intent, with the phrase ***turn the fan off***. Then connect the word "fan" to the **device** entity.

### Train and Test the Language Model

Now you're ready to use the data you've provided in the form of entities, intents, and uterances to train the language model for your LUIS app.

1. At the top of the LUIS page for your app, click **Train** to train the language model
2. After the app has been trained, click **Test**, and then in the test pane, enter the following utterances and inspect the results to verify that they are correctly interpreted as commands for the *Switch On* and *Switch Off* intents with the appropriate entities:
    * *turn on the light*
    * *switch the light off*
    * *switch the fan on*
    * *turn off the fan*
    
### Publish the Model and Configure Endpoints

To use your trained model in a client application, you must publish it as an endpoint to which the client applications can send new utterances; from which intents and entitites will be predicted.

1. At the top of the LUIS page for your app, click **Publish**. Then select **Production slot** and click **Done**.
2. After the model has been published, at the top of the LUIS page for your app, click **Manage**. Then on the **Application Information** tab, note the **Application ID** for your app. Copy this and paste it in the code below to replace **YOUR_LUIS_APP_ID**.
3. On the **Azure Resources** tab, note the **Primary key** and **Endpoint URL** for your language model, which is based on your cognitive services resource. Copy these and paste them into the code below, replacing **YOUR_COG_KEY** and **YOUR_COG_ENDPOINT**.
4. Run the cell below, and when prompted (at the top of this page), enter the text *turn the light on*. The text is interpreted by your LUIS model and an appropriate image is displayed.


In [ ]:
from scripts import office_auto
import matplotlib.pyplot as plt
from PIL import Image
import os
%matplotlib inline

try:
    # Set up API configuration
    luis_app_id = 'YOUR_LUIS_APP_ID'
    cog_key = 'YOUR_COG_KEY'
    cog_endpoint = 'YOUR_COG_ENDPOINT'

    # prompt for a command
    command = input('Please enter a command: \n')

    # get the predicted intent and entity
    img_name = office_auto.get_intent(luis_app_id, cog_key, cog_endpoint, command)

    # display an appropriate image
    img = Image.open(os.path.join("data", "office-auto" ,img_name))
    plt.axis('off')
    plt. imshow(img)
except Exception as ex:
    print(ex)

Re-run the cell above, trying the following phrases:

* *turn on the light*
* *put the lamp off*
* *switch the fan on*
* *switch the light on*
* *switch off the light*
* *turn off the fan*
* *switch the AC on*

> **Note**: If you're curious about the code used to retrieve the intents and entitites from your LUIS app, look at the **office_auto.py** file in the **scripts** folder.

## Using the Speech Cognitive Service

So far, we've seen how analyze text; but increasingly AI systems enable humans to communicate with software services through speech recognition. To support this, the **Speech** cognitive service provides a simple way to transcribe spoken language into text.

In the cell below, paste the LUIS Application ID, Cognitive Services key, and Cognitive Services endpoint you've used previously. Then run the cell.

This code will use the Speech cognitive service to transcribe the **light-on.wav** audio file, and then use LUIS to determine the intent of the transcribed speech.

In [ ]:
from scripts import office_auto
import os
import IPython
import os
from azure.cognitiveservices.speech import SpeechConfig, SpeechRecognizer, AudioConfig

try:
    # Set up API configuration
    luis_app_id = 'YOUR_LUIS_APP_ID'
    cog_key = 'YOUR_COG_KEY'
    cog_endpoint = 'YOUR_COG_ENDPOINT'
    
    # Get region from endpoint
    cog_region = cog_endpoint[8:cog_endpoint.find('.')]

    # Get spoken command
    file_name = 'light-on.wav'
    audio_file = os.path.join('data', 'office-auto', file_name)

    # Configure speech recognizer
    speech_config = SpeechConfig(cog_key, cog_region)
    audio_config = AudioConfig(filename=audio_file) #Use file instead of default (microphone)
    speech_recognizer = SpeechRecognizer(speech_config, audio_config)

    # Use a one-time, synchronous call to transcribe the speech
    speech = speech_recognizer.recognize_once()

    # get the predicted intent and entity
    img_name = office_auto.get_intent(luis_app_id, cog_key, cog_endpoint, speech.text)

    # Play audio and display image
    IPython.display.display(IPython.display.Audio(audio_file, autoplay=True),
                            IPython.display.Image(data=os.path.join("data", "office-auto" ,img_name)))
except Exception as ex:
    print(ex)

Try modifying the cell above to use the **light-off.wav** audio file.